# ਨਾਂਦਿਤ ਇਕਾਈ ਪਛਾਣ (NER)

ਇਹ ਨੋਟਬੁੱਕ [AI for Beginners Curriculum](http://aka.ms/ai-beginners) ਤੋਂ ਹੈ।

ਇਸ ਉਦਾਹਰਨ ਵਿੱਚ, ਅਸੀਂ ਸਿੱਖਾਂਗੇ ਕਿ [Annotated Corpus for Named Entity Recognition](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus) ਡੇਟਾਸੈੱਟ ਤੋਂ Kaggle 'ਤੇ NER ਮਾਡਲ ਨੂੰ ਕਿਵੇਂ ਟ੍ਰੇਨ ਕਰਨਾ ਹੈ। ਅੱਗੇ ਵਧਣ ਤੋਂ ਪਹਿਲਾਂ, ਕਿਰਪਾ ਕਰਕੇ [ner_dataset.csv](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus?resource=download&select=ner_dataset.csv) ਫਾਈਲ ਨੂੰ ਮੌਜੂਦਾ ਡਾਇਰੈਕਟਰੀ ਵਿੱਚ ਡਾਊਨਲੋਡ ਕਰੋ।


In [62]:
import pandas as pd
from tensorflow import keras
import numpy as np

## ਡਾਟਾਸੈੱਟ ਤਿਆਰ ਕਰਨਾ

ਅਸੀਂ ਡਾਟਾਸੈੱਟ ਨੂੰ ਇੱਕ ਡਾਟਾਫਰੇਮ ਵਿੱਚ ਪੜ੍ਹਨ ਨਾਲ ਸ਼ੁਰੂ ਕਰਾਂਗੇ। ਜੇ ਤੁਸੀਂ Pandas ਦੀ ਵਰਤੋਂ ਬਾਰੇ ਹੋਰ ਜਾਣਨਾ ਚਾਹੁੰਦੇ ਹੋ, ਤਾਂ ਸਾਡੇ [Data Science for Beginners](http://aka.ms/datascience-beginners) ਵਿੱਚ [ਡਾਟਾ ਪ੍ਰੋਸੈਸਿੰਗ ਦੇ ਪਾਠ](https://github.com/microsoft/Data-Science-For-Beginners/tree/main/2-Working-With-Data/07-python) 'ਤੇ ਜਾਓ।


In [3]:
df = pd.read_csv('ner_dataset.csv',encoding='unicode-escape')
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


ਆਓ ਵਿਲੱਖਣ ਟੈਗ ਪ੍ਰਾਪਤ ਕਰੀਏ ਅਤੇ ਲੁੱਕਅੱਪ ਡਿਕਸ਼ਨਰੀ ਬਣਾਈਏ ਜੋ ਅਸੀਂ ਟੈਗ ਨੂੰ ਕਲਾਸ ਨੰਬਰਾਂ ਵਿੱਚ ਤਬਦੀਲ ਕਰਨ ਲਈ ਵਰਤ ਸਕੀਏ:


In [4]:
tags = df.Tag.unique()
tags

array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat'], dtype=object)

In [8]:
id2tag = dict(enumerate(tags))
tag2id = { v : k for k,v in id2tag.items() }

id2tag[0]

'O'

ਹੁਣ ਸਾਨੂੰ ਸ਼ਬਦਾਵਲੀ ਨਾਲ ਵੀ ਇਹੀ ਕਰਨਾ ਹੈ। ਸੌਖੇਪਣ ਲਈ, ਅਸੀਂ ਸ਼ਬਦਾਵਲੀ ਨੂੰ ਬਿਨਾਂ ਸ਼ਬਦਾਂ ਦੀ ਆਵ੍ਰਿਤੀ ਨੂੰ ਧਿਆਨ ਵਿੱਚ ਲਿਆਂਦੇ ਬਣਾਵਾਂਗੇ; ਅਸਲ ਜ਼ਿੰਦਗੀ ਵਿੱਚ ਤੁਸੀਂ Keras ਵੈਕਟਰਾਈਜ਼ਰ ਦੀ ਵਰਤੋਂ ਕਰ ਸਕਦੇ ਹੋ ਅਤੇ ਸ਼ਬਦਾਂ ਦੀ ਗਿਣਤੀ ਨੂੰ ਸੀਮਿਤ ਕਰ ਸਕਦੇ ਹੋ।


In [14]:
vocab = set(df['Word'].apply(lambda x: x.lower()))
id2word = { i+1 : v for i,v in enumerate(vocab) }
id2word[0] = '<UNK>'
vocab.add('<UNK>')
word2id = { v : k for k,v in id2word.items() }

ਅਸੀਂ ਸਿਖਲਾਈ ਲਈ ਵਾਕਾਂ ਦਾ ਡੇਟਾਸੈੱਟ ਬਣਾਉਣ ਦੀ ਲੋੜ ਹੈ। ਆਓ ਅਸਲ ਡੇਟਾਸੈੱਟ ਵਿੱਚੋਂ ਗੁਜ਼ਰ ਕੇ ਸਾਰੇ ਵੱਖ-ਵੱਖ ਵਾਕਾਂ ਨੂੰ `X` (ਸ਼ਬਦਾਂ ਦੀਆਂ ਸੂਚੀਆਂ) ਅਤੇ `Y` (ਟੋਕਨ ਦੀ ਸੂਚੀ) ਵਿੱਚ ਵੰਡ ਲਈਏ:


In [41]:
X,Y = [],[]
s,t = [],[]
for i,row in df[['Sentence #','Word','Tag']].iterrows():
    if pd.isna(row['Sentence #']):
        s.append(row['Word'])
        t.append(row['Tag'])
    else:
        if len(s)>0:
            X.append(s)
            Y.append(t)
        s,t = [row['Word']],[row['Tag']]
X.append(s)
Y.append(t)


In [93]:
def vectorize(seq):
    return [word2id[x.lower()] for x in seq]

def tagify(seq):
    return [tag2id[x] for x in seq]

Xv = list(map(vectorize,X))
Yv = list(map(tagify,Y))

Xv[0], Yv[0]

([10386,
  23515,
  4134,
  29620,
  7954,
  13583,
  21193,
  12222,
  27322,
  18258,
  5815,
  15880,
  5355,
  25242,
  31327,
  18258,
  27067,
  23515,
  26444,
  14412,
  358,
  26551,
  5011,
  30558],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0])

ਸਰਲਤਾ ਲਈ, ਅਸੀਂ ਸਾਰੀਆਂ ਵਾਕਾਂ ਨੂੰ ਵੱਧ ਤੋਂ ਵੱਧ ਲੰਬਾਈ ਤੱਕ 0 ਟੋਕਨ ਨਾਲ ਭਰ ਦਿਆਂਗੇ। ਅਸਲ ਜ਼ਿੰਦਗੀ ਵਿੱਚ, ਅਸੀਂ ਹੋਰ ਚਤੁਰਾਈ ਭਰੀ ਰਣਨੀਤੀ ਵਰਤਣਾ ਚਾਹਾਂਗੇ ਅਤੇ ਸਿਰਫ਼ ਇੱਕ ਮਿਨੀਬੈਚ ਦੇ ਅੰਦਰ ਕ੍ਰਮਾਂ ਨੂੰ ਭਰਨਾ ਚਾਹਾਂਗੇ।


In [51]:
X_data = keras.preprocessing.sequence.pad_sequences(Xv,padding='post')
Y_data = keras.preprocessing.sequence.pad_sequences(Yv,padding='post')

## ਟੋਕਨ ਵਰਗੀਕਰਨ ਨੈੱਟਵਰਕ ਦੀ ਪਰਿਭਾਸ਼ਾ

ਅਸੀਂ ਟੋਕਨ ਵਰਗੀਕਰਨ ਲਈ ਦੋ-ਪਰਤਾਂ ਵਾਲੇ ਬਿਡਾਇਰੈਕਸ਼ਨਲ LSTM ਨੈੱਟਵਰਕ ਦੀ ਵਰਤੋਂ ਕਰਾਂਗੇ। ਆਖਰੀ LSTM ਪਰਤ ਦੇ ਹਰ ਆਉਟਪੁੱਟ 'ਤੇ ਡੈਂਸ ਵਰਗੀਕਰਨ ਲਾਗੂ ਕਰਨ ਲਈ, ਅਸੀਂ `TimeDistributed` ਕਨਸਟਰਕਸ਼ਨ ਦੀ ਵਰਤੋਂ ਕਰਾਂਗੇ, ਜੋ LSTM ਦੇ ਹਰ ਕਦਮ 'ਤੇ ਇੱਕੋ ਜਿਹੇ ਡੈਂਸ ਪਰਤ ਨੂੰ ਦੁਹਰਾਉਂਦਾ ਹੈ:


In [94]:
maxlen = X_data.shape[1]
vocab_size = len(vocab)
num_tags = len(tags)
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, 300, input_length=maxlen),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.TimeDistributed(keras.layers.Dense(num_tags, activation='softmax'))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 104, 300)          9545400   
                                                                 
 bidirectional_6 (Bidirectio  (None, 104, 200)         320800    
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 104, 200)         240800    
 nal)                                                            
                                                                 
 time_distributed_3 (TimeDis  (None, 104, 17)          3417      
 tributed)                                                       
                                                                 
Total params: 10,110,417
Trainable params: 10,110,417
Non-trainable params: 0
__________________________________________

ਇੱਥੇ ਧਿਆਨ ਦਿਓ ਕਿ ਅਸੀਂ ਆਪਣੇ ਡਾਟਾਸੈੱਟ ਲਈ `maxlen` ਸਪਸ਼ਟ ਤੌਰ 'ਤੇ ਨਿਰਧਾਰਤ ਕਰ ਰਹੇ ਹਾਂ - ਜੇਕਰ ਅਸੀਂ ਚਾਹੁੰਦੇ ਹਾਂ ਕਿ ਨੈੱਟਵਰਕ ਵੱਖ-ਵੱਖ ਲੰਬਾਈ ਵਾਲੀਆਂ ਲੜੀਆਂ ਨੂੰ ਸੰਭਾਲ ਸਕੇ, ਤਾਂ ਸਾਨੂੰ ਨੈੱਟਵਰਕ ਨੂੰ ਨਿਰਧਾਰਤ ਕਰਦੇ ਸਮੇਂ ਕੁਝ ਹੋਰ ਚੁਸਤ ਹੋਣਾ ਪਵੇਗਾ।

ਚਲੋ ਹੁਣ ਮਾਡਲ ਨੂੰ ਟ੍ਰੇਨ ਕਰਦੇ ਹਾਂ। ਗਤੀ ਨੂੰ ਤੇਜ਼ ਕਰਨ ਲਈ, ਅਸੀਂ ਸਿਰਫ ਇੱਕ epoch ਲਈ ਟ੍ਰੇਨ ਕਰਾਂਗੇ, ਪਰ ਤੁਸੀਂ ਲੰਬੇ ਸਮੇਂ ਲਈ ਟ੍ਰੇਨ ਕਰਨ ਦੀ ਕੋਸ਼ਿਸ਼ ਕਰ ਸਕਦੇ ਹੋ। ਇਸ ਤੋਂ ਇਲਾਵਾ, ਤੁਸੀਂ ਡਾਟਾਸੈੱਟ ਦੇ ਕੁਝ ਹਿੱਸੇ ਨੂੰ ਟ੍ਰੇਨਿੰਗ ਡਾਟਾਸੈੱਟ ਵਜੋਂ ਵੱਖ ਕਰ ਸਕਦੇ ਹੋ, ਤਾਂ ਜੋ ਵੈਲੀਡੇਸ਼ਨ ਐਕ੍ਯੂਰੇਸੀ ਦਾ ਅਧਿਐਨ ਕੀਤਾ ਜਾ ਸਕੇ।


In [57]:
model.fit(X_data,Y_data)

1499/1499 [==============================] - 740s 488ms/step - loss: 0.0667 - acc: 0.9841


## ਨਤੀਜੇ ਦੀ ਜਾਂਚ

ਆਓ ਹੁਣ ਦੇਖੀਏ ਕਿ ਸਾਡਾ ਐਨਟੀਟੀ ਪਛਾਣ ਮਾਡਲ ਇੱਕ ਨਮੂਨਾ ਵਾਕ 'ਤੇ ਕਿਵੇਂ ਕੰਮ ਕਰਦਾ ਹੈ:


In [91]:
sent = 'John Smith went to Paris to attend a conference in cancer development institute'
words = sent.lower().split()
v = keras.preprocessing.sequence.pad_sequences([[word2id[x] for x in words]],padding='post',maxlen=maxlen)
res = model(v)[0]

In [92]:
r = np.argmax(res.numpy(),axis=1)
for i,w in zip(r,words):
    print(f"{w} -> {id2tag[i]}")

john -> B-per
smith -> I-per
went -> O
to -> O
paris -> B-geo
to -> O
attend -> O
a -> O
conference -> O
in -> O
cancer -> B-org
development -> I-org
institute -> I-org


## ਮੁੱਖ ਗੱਲ

ਇੱਕ ਸਧਾਰਨ LSTM ਮਾਡਲ ਵੀ NER ਵਿੱਚ ਠੀਕ ਨਤੀਜੇ ਦਿਖਾਉਂਦਾ ਹੈ। ਹਾਲਾਂਕਿ, ਹੋਰ ਵਧੀਆ ਨਤੀਜੇ ਪ੍ਰਾਪਤ ਕਰਨ ਲਈ, ਤੁਸੀਂ ਵੱਡੇ ਪੂਰਵ-ਪ੍ਰਸ਼ਿਕਸ਼ਿਤ ਭਾਸ਼ਾ ਮਾਡਲਾਂ ਜਿਵੇਂ ਕਿ BERT ਦੀ ਵਰਤੋਂ ਕਰ ਸਕਦੇ ਹੋ। Huggingface Transformers ਲਾਇਬ੍ਰੇਰੀ ਦੀ ਵਰਤੋਂ ਕਰਕੇ NER ਲਈ BERT ਨੂੰ ਪ੍ਰਸ਼ਿਕਸ਼ਿਤ ਕਰਨ ਦੀ ਵਿਵਰਣਾ [ਇੱਥੇ](https://huggingface.co/course/chapter7/2?fw=pt) ਦਿੱਤੀ ਗਈ ਹੈ।



---

**ਅਸਵੀਕਰਤੀ**:  
ਇਹ ਦਸਤਾਵੇਜ਼ AI ਅਨੁਵਾਦ ਸੇਵਾ [Co-op Translator](https://github.com/Azure/co-op-translator) ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਅਨੁਵਾਦ ਕੀਤਾ ਗਿਆ ਹੈ। ਜਦੋਂ ਕਿ ਅਸੀਂ ਸਹੀ ਹੋਣ ਦਾ ਯਤਨ ਕਰਦੇ ਹਾਂ, ਕਿਰਪਾ ਕਰਕੇ ਧਿਆਨ ਦਿਓ ਕਿ ਸਵੈਚਾਲਿਤ ਅਨੁਵਾਦਾਂ ਵਿੱਚ ਗਲਤੀਆਂ ਜਾਂ ਅਸੁੱਤੀਆਂ ਹੋ ਸਕਦੀਆਂ ਹਨ। ਇਸ ਦੀ ਮੂਲ ਭਾਸ਼ਾ ਵਿੱਚ ਲਿਖਿਆ ਦਸਤਾਵੇਜ਼ ਪ੍ਰਮਾਣਿਕ ਸਰੋਤ ਮੰਨਿਆ ਜਾਣਾ ਚਾਹੀਦਾ ਹੈ। ਮਹੱਤਵਪੂਰਨ ਜਾਣਕਾਰੀ ਲਈ, ਪੇਸ਼ੇਵਰ ਮਨੁੱਖੀ ਅਨੁਵਾਦ ਦੀ ਸਿਫਾਰਸ਼ ਕੀਤੀ ਜਾਂਦੀ ਹੈ। ਇਸ ਅਨੁਵਾਦ ਦੇ ਪ੍ਰਯੋਗ ਤੋਂ ਪੈਦਾ ਹੋਣ ਵਾਲੇ ਕਿਸੇ ਵੀ ਗਲਤਫਹਿਮੀ ਜਾਂ ਗਲਤ ਵਿਆਖਿਆ ਲਈ ਅਸੀਂ ਜ਼ਿੰਮੇਵਾਰ ਨਹੀਂ ਹਾਂ।  
